<a href="https://colab.research.google.com/github/ekkolabs/stylegan2-ada/blob/main/ColabNotebook-Bosch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Training StyleGan2-ADA

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [ ]:
%tensorflow_version 1.x

In [ ]:
!nvidia-smi

## Install Repo to Google Drive

Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [ ]:
#import os
#if os.path.isdir("/content/drive/My Drive/colab-sg2-ada"):
#    %cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
#else:
if 1:
    #install script
    %cd "/content/drive/My Drive/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git checkout origin/main -- train.py

In [ ]:
#IF RAW STUFF!git checkout 6d8f14ed886dc969f1fb5c68aeafb93e34836f2b
%cd "/content/drive/My Drive/colab-sg2-ada/"
#!git clone https://github.com/dvschultz/stylegan2-ada
%cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
!git checkout 6d8f14ed886dc969f1fb5c68aeafb93e34836f2b


## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [ ]:
#if you manually uploaded your dataset to Colab, unzip it
zip_path = "/content/drive/My\ Drive/mlart/pera.zip"

!unzip {zip_path} -d "/content/drive/My Drive/mlart/pera/"

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [ ]:
!pip install python-resize-image
#update this to the path to your image folder
source_path = "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada/datasets/bosch/"








from PIL import Image

from resizeimage import resizeimage


for f in os.listdir(source_path):
  if f.endswith('.jpg'):
    with open(os.path.join(source_path,f), 'r+b') as f1:
        with Image.open(f1) as image:
            try:
              cover = resizeimage.resize_crop(image, [1024, 1024])
            except:
              print(f)
              os.remove(os.path.join(source_path,f))
              cover.save(os.path.join(source_path,f), image.format)

In [ ]:
#
#display image size:
#
dataset_path = "/content/drive/My Drive/mlart/ando"

import cv2


for i,f in enumerate(os.listdir(dataset_path)):
  im = cv2.imread(os.path.join(dataset_path,f))
  #print(f,im.shape)
  w,h,c = im.shape
  if w != 1024 or h != 1024:
      print(f)

In [ ]:
#os.remove('/content/drive/My Drive/mlart/pera/jquery.min.js.download')
#!rm -rf "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada-2"
#!rm -rf "./datasets/ando"
#!cp -r "/content/drive/My Drive/mlart/ando" "/content/drive/My Drive/mlart/ando2" 



In [ ]:
import os
dataset_path = "/content/drive/My Drive/mlart/ando"

for i,f in enumerate(os.listdir(dataset_path)):
  os.rename(os.path.join(dataset_path,f),os.path.join(dataset_path,str(i)+".jpg"))


In [ ]:
#update this to the path to your image folder
dataset_path = "/content/drive/My\ Drive/mlart/ando"
#give your dataset a name
dataset_name = "ando"

#you don't need to edit anything here
!python dataset_tool.py create_from_images_raw ./datasets/{dataset_name} {dataset_path}

## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [ ]:
!python train.py --help

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "bosch"
#how often should the model generate samples and a .pkl file
snapshot_count = 4
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = "bg"

#raw
use_raw = True

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
#resume_from = "ffhq1024"
#resume_from = "ffhq1024" #
resume_from = "/content/drive/My\ Drive/colab-sg2-ada/stylegan2-ada/results/00005-bosch-mirror-11gb-gpu-bg-resumeffhq1024/network-snapshot-000064.pkl"
#"/content/drive/My\ Drive/colab-sg2-ada/stylegan2-ada/results/00005-pera-mirror-11gb-gpu-bg-resumecustom/network-snapshot-000032.pkl"
#DEFAULT CFG
config = "11gb-gpu"


#don't edit this unless you know what you're doing :)

!python train.py --outdir ./results --snap={snapshot_count} --cfg={config} --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from} --use-raw={use_raw}

### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.

In [ ]:
!python generate.py --help

In [ ]:
!python generate.py lerp-video --outdir=out --seeds=0-40  --network="/content/drive/My Drive/colab-sg2-ada/stylegan2-ada/results/00006-pera-mirror-11gb-gpu-bg-resumecustom/network-snapshot-000256.pkl"

In [ ]:
!python generate.py --outdir=out --dlatents=out/dlatents.npz --network="/content/drive/My Drive/colab-sg2-ada/stylegan2-ada/results/00006-pera-mirror-11gb-gpu-bg-resumecustom/network-snapshot-000256.pkl"

In [ ]:
!python generate.py generate-latent-walk --outdir=out --seeds=0-10  --network="/content/drive/My Drive/colab-sg2-ada/stylegan2-ada/results/00006-pera-mirror-11gb-gpu-bg-resumecustom/network-snapshot-000256.pkl"

In [ ]:
!python generate.py truncation-traversal --trunc=1 --outdir=out --network="/content/drive/My Drive/colab-sg2-ada/stylegan2-ada/results/00006-pera-mirror-11gb-gpu-bg-resumecustom/network-snapshot-000256.pkl"

In [ ]:
!python aydao_flesh_digressions.py --pkl "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada/results/00006-pera-mirror-11gb-gpu-bg-resumecustom/network-snapshot-000256.pkl"

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"

network_pkl="/content/drive/My\ Drive/mlart/pkl/bosch-ffhq-128.pkl"

!pip install opensimplex

!python generate.py generate-latent-walk --trunc=0.8 --frames=240 --seeds=83,72,83 --walk-type="line-w" --network={network_pkl}
